In [0]:
import sqlite3
from sqlite3 import Error
import pandas as pd

In [0]:
def setup_task(queries):
  """ Create database residing in RAM,
  create tables with sql queries,
  return db connection. """
  conn = None
  try:
    conn = sqlite3.connect(':memory:')
    cur = conn.cursor()
    for query in queries:
      cur.execute(query)     
  except Error as e:
    print(e)
  else:
    print("DB setup has been successfully completed!")
    return conn

# Task 1: Departments' salaries

## Setup task 1

In [0]:
create_departments =  """ CREATE TABLE IF NOT EXISTS departments (
                                        id integer PRIMARY KEY,
                                        name text NOT NULL                                        
                                        );
                      """

In [0]:
insert_departments =  """ INSERT INTO departments (id, name)
                          VALUES
	                        ("1", "IT"),
                          ("2", "Sales");	
                      """

In [0]:
create_employees =  """ CREATE TABLE IF NOT EXISTS employees (
                                        id integer PRIMARY KEY,
                                        name text NOT NULL,
                                        salary integer NOT NULL,
                                        department_id integer NOT NULL,
                                        FOREIGN KEY(department_id) REFERENCES departments(id)                                        
                                        );
                    """

In [0]:
insert_employees =  """ INSERT INTO employees (id, name, salary, department_id)
                        VALUES
	                      ("1", "Joe", "70000", "1"),
                        ("2", "Henry", "80000", "2"),
                        ("3", "Sam", "60000", "2"),
                        ("4", "Max", "90000", "1");
                    """

In [0]:
task_1_conn = setup_task([create_departments, insert_departments, create_employees, insert_employees])

DB setup has been successfully completed!


## Solutions task 1 (SQL)

1.1. Напишите SQL запрос, который найдет самые большие зарплаты для каждого департамента


In [0]:
pd.read_sql_query(""" SELECT department_id, t1.name, MAX(salary) AS max_salary
                      FROM departments t1
                      INNER JOIN employees t2
                      ON t1.id = t2.department_id
                      GROUP BY department_id; """,
                  task_1_conn,
                  index_col="department_id")

,name,max_salary
department_id,,
1,IT,90000
2,Sales,80000


1.2. Напишите SQL запрос, который найдет среднюю зарплату для каждого департамента, исключая департаменты, в кот. средняя зарплата равна или ниже 7000

In [0]:
pd.read_sql_query(""" SELECT department_id, t1.name, AVG(salary) AS avg_salary
                      FROM departments t1
                      INNER JOIN employees t2
                      ON t1.id = t2.department_id
                      GROUP BY department_id
                      HAVING AVG(salary) > 70000; """,
                  task_1_conn,
                  index_col="department_id")

,name,avg_salary
department_id,,
1,IT,80000.0


1.3. Напишите SQL запрос, чтобы найти вторую самую высокую зарплату работника

In [0]:
pd.read_sql_query(""" SELECT MAX(salary) AS second_max_salary
                      FROM employees
                      WHERE salary < (SELECT MAX(salary)
                                      FROM employees); """,
                  task_1_conn)

,second_max_salary
0,80000


1.4. Сделать несколько вариантов для пункта 1.3 и провести профайлинг и выбрать какой запрос лучше. Как вы думаете почему



In [0]:
pd.read_sql_query(""" SELECT salary AS second_max_salary
                      FROM employees
                      ORDER BY salary DESC
                      LIMIT 1, 1; """,
                  task_1_conn)

,second_max_salary
0,80000


Время работы алгоритмов п. 1.3 и 1.4: 

- для п.1.4: **O(n) = N^2** (для большинства алгоритмов сортировки)
- для п.1.3: **O(n) = 3*N**: N (поиск максимального значения по исходному массиву) + N (отбор элементов меньше максимального значения) + N (выбор максимального значения из офильтрованого массива)

In [0]:
prof_1_3 = (task_1_conn
          .execute(
          """ EXPLAIN QUERY PLAN
                SELECT MAX(salary) AS second_max_salary
                FROM employees
                WHERE salary < (SELECT MAX(salary)
                                FROM employees);""")
          .fetchall())

for el in prof_1_3:
  print(el, "\n")

(0, 0, 0, 'SEARCH TABLE employees') 

(0, 0, 0, 'EXECUTE SCALAR SUBQUERY 1') 

(1, 0, 0, 'SEARCH TABLE employees') 



In [0]:
prof_1_4 = (task_1_conn
          .execute(
              """ EXPLAIN QUERY PLAN
                    SELECT salary AS second_max_salary
                    FROM employees
                    ORDER BY salary DESC
                    LIMIT 1, 1; """)
          .fetchall())

for el in prof_1_4:
  print(el, "\n")

(0, 0, 0, 'SCAN TABLE employees') 

(0, 0, 0, 'USE TEMP B-TREE FOR ORDER BY') 



* SEARCH - проход по отдельным строкам таблицы
* SCAN - проход по всей таблице 

## Solutions task 1 (pandas)

In [0]:
deps = pd.read_sql_query("SELECT * FROM departments", task_1_conn, index_col="id")
emps = pd.read_sql_query("SELECT * FROM employees", task_1_conn, index_col="id")
task_1_conn.close()

In [0]:
comb = deps.merge(emps, left_on=deps.index, right_on='department_id')

In [0]:
comb.groupby("name_x")["salary"].max()

name_x
IT       90000
Sales    80000
Name: salary, dtype: int64

In [0]:
comb.groupby("name_x")["salary"].mean().where(lambda x: x > 70000).dropna()

name_x
IT    80000.0
Name: salary, dtype: float64

In [0]:
%%timeit
emps_sort = emps.sort_values(by="salary")
emps_sort.loc[emps_sort.index[-2], "salary"]

1000 loops, best of 3: 432 µs per loop


In [0]:
%timeit emps.salary.nlargest(2).iloc[1]

The slowest run took 4.60 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 271 µs per loop


# Task 2: Employees and managers

## Setup task 2

In [0]:
create_employees_2 =  """ CREATE TABLE IF NOT EXISTS employees (
                                        id integer PRIMARY KEY,
                                        name text NOT NULL,
                                        salary integer NOT NULL,
                                        manager_id integer NOT NULL,                                        
                                        sex text NOT NULL,
                                        empl_date text NOT NULL,
                                        FOREIGN KEY(manager_id) REFERENCES employees(id) 
                                        );
                      """

In [0]:
insert_employees_2 =  """ INSERT INTO employees (id, name, salary, manager_id, sex, empl_date)
                          VALUES
	                        ("1", "Joe", "70000", "3", "m", "20190301"),
                          ("2", "Henry", "80000", "4", "m", "20190401"),
                          ("3", "Sam", "60000", "NULL", "s", "20190501"),
                          ("4", "Max", "90000", "NULL", "m", "20190101");
                      """

In [0]:
task_2_conn = setup_task([create_employees_2, insert_employees_2])

DB setup has been successfully completed!


## Solutions task 2 (SQL)

2.1. Напишите SQL запрос который найдет имена всех работников, которые получают больше чем их менеджеры. Если у работника нет менеджера, они не должны попадать в выборку.


In [0]:
pd.read_sql_query(""" SELECT t1.name
                      FROM employees t1
                      INNER JOIN employees t2
                      ON t1.manager_id = t2.id
                      WHERE t1.salary > t2.salary; """,
                  task_2_conn)

,name
0,Joe


2.2. Написать SQL запрос, где найти количество сотрудников в зависимости от пола и принятые на работу в диапазоне между 01/02/2019 и 31/11/2019.


In [0]:
pd.read_sql_query(""" SELECT sex, COUNT(*) AS num_employees
                      FROM employees
                      WHERE empl_date BETWEEN '20190201' AND '20191131'
                      GROUP BY sex; """,
                  task_2_conn)

,sex,num_employees
0,m,2
1,s,1


2.3. Напишите SQL запрос, чтобы найти список сотрудников, в именах которых присутствует буква 'M' или 'm'


In [0]:
pd.read_sql_query(""" SELECT *
                      FROM employees
                      WHERE name LIKE '%m%'; """,
                  task_2_conn)

,id,name,salary,manager_id,sex,empl_date
0,3,Sam,60000,NULL,s,20190501
1,4,Max,90000,NULL,m,20190101


## Solutions task 2 (pandas)

In [0]:
emp = pd.read_sql_query("SELECT * FROM employees", task_2_conn, index_col="id")
task_2_conn.close()

In [0]:
comb_self = pd.merge(emp, emp, left_on=emp.index, right_on="manager_id")
comb_self.loc[comb_self["salary_y"] > comb_self["salary_x"], "name_y"]

0    Joe
Name: name_y, dtype: object

In [0]:
emp_fil = emp.loc[(emp["empl_date"] > "20190201") & (emp["empl_date"] < "20191131")]
emp_fil.groupby("sex")["name"].count()

sex
m    2
s    1
Name: name, dtype: int64

In [0]:
emp[emp["name"].str.contains("m|M")]

,name,salary,manager_id,sex,empl_date
id,,,,,
3,Sam,60000,NULL,s,20190501
4,Max,90000,NULL,m,20190101


# Task 3: Temperature

## Setup task 3

In [0]:
create_weathers = """ CREATE TABLE IF NOT EXISTS weathers (
                                      id integer PRIMARY KEY,
                                      date text NOT NULL,
                                      temperature integer NOT NULL
                                      );
                  """

In [0]:
insert_weathers = """ INSERT INTO weathers (id, date, temperature)
                          VALUES
	                        ("1", "20160101", "10"),
                          ("2", "20160102", "25"),
                          ("3", "20160103", "20"),
                          ("4", "20160104", "30");
                  """

In [0]:
task_3_conn = setup_task([create_weathers, insert_weathers])

DB setup has been successfully completed!


## Solutions task 3 (SQL)

3.1. Напишите SQL запрос для выбора id всех записей, температура в которых была выше, чем в предыдущий день (чем вчера)

In [0]:
# LAG () is not supported in current version of sqlite

# SELECT id
# FROM  (SELECT id,
#               temperature - LAG(temperature, 1)
#                             OVER (ORDER BY date) AS change
#       FROM weathers) t1
# WHERE change > 0

In [0]:
pd.read_sql_query(""" SELECT id                      
                      FROM weathers t1
                      WHERE temperature - (SELECT t2.temperature 
                                          FROM weathers t2 
                                          WHERE 
                                          t2.date < t1.date      
                                          ORDER BY t2.date DESC
                                          LIMIT 1
                                          ) > 0; """,
                  task_3_conn)

,id
0,2
1,4


## Solutions task 3 (pandas)

In [0]:
weathers = pd.read_sql_query("SELECT * FROM weathers", task_3_conn, index_col="id")
task_3_conn.close()

In [0]:
weathers["date"] = pd.to_datetime(weathers["date"])

In [0]:
weathers = weathers.sort_values(by="date")
weathers["prev_temperature"] = weathers["temperature"].shift(1)
list(weathers.index[weathers["temperature"] > weathers["prev_temperature"]])

[2, 4]

# Task 4: Grades

## Setup task 4

In [0]:
create_tests =  """ CREATE TABLE IF NOT EXISTS tests (
                                      id integer PRIMARY KEY,
                                      description text NOT NULL                                      
                                      );
                """

In [0]:
insert_tests =  """ INSERT INTO tests (id, description)
                          VALUES
	                        ("1", "test_1"),
                          ("2", "test_2"),
                          ("3", "test_3"),
                          ("4", "test_4");
                """

In [0]:
create_grades = """ CREATE TABLE IF NOT EXISTS grades (
                                      id integer PRIMARY KEY,
                                      test_id integer NOT NULL,
                                      grade text,
                                      grader integer
                                      );
                """

In [0]:
insert_grades = """ INSERT INTO grades (id, test_id, grade, grader)
                          VALUES
	                        ("1", "1", "A", "1"),
                          ("2", "1", "B", "2"),
                          ("3", "2", "C", "2"),
                          ("4", "2", "C", "3"),
                          ("5", "3", "C", "1"),
                          ("6", "3", "D", "3"),
                          ("7", "4", NULL, NULL);
                """

In [0]:
task_4_conn = setup_task([create_tests, insert_tests, create_grades, insert_grades])

DB setup has been successfully completed!


## Solutions task 4 (SQL)

4.1. Написать SQL запрос для получения списка всех тестов, которые не были оценены grader=”grader1”.


In [0]:
pd.read_sql_query(""" SELECT id, description
                      FROM tests
                      WHERE id NOT IN (SELECT test_id
                                      FROM grades
                                  WHERE grader = 1)
                        OR id IN (SELECT test_id
                                  FROM grades
                                  WHERE grader is NULL); """,
                  task_4_conn,
                  index_col="id")

,description
id,
2,test_2
4,test_4


## Solutions task 4 (pandas)

In [0]:
tests = pd.read_sql_query("SELECT * FROM tests", task_4_conn, index_col="id")
grades = pd.read_sql_query("SELECT * FROM grades", task_4_conn, index_col="id")
task_4_conn.close()

In [0]:
tests_by_grader_1 = (grades
                     .groupby("test_id")
                     .filter(lambda g: any(g.grader == 1))["test_id"]
                     .unique())

tests.loc[~tests.index.isin(tests_by_grader_1), :]

,description
id,
2,test_2
4,test_4


# Task 5: Students

## Setup task 5

In [0]:
create_students =   """ CREATE TABLE IF NOT EXISTS students (
                                      student_id integer PRIMARY KEY,
                                      mark integer NOT NULL                                      
                                      );
                    """

In [0]:
insert_students =   """ INSERT INTO students (student_id, mark)
                          VALUES
	                        ("1", "5"),
                          ("2", "4"),
                          ("3", "4"),
                          ("4", "4"),
                          ("5", "3"),
                          ("6", "3"),
                          ("7", "5");
                    """

In [0]:
task_5_conn = setup_task([create_students, insert_students])

DB setup has been successfully completed!


## Solutions task 5 (SQL)

5.1. Найти всех студентов, чьи оценки являются больше, чем средние оценки, т.е. список студентов выше среднего

In [0]:
pd.read_sql_query(""" SELECT student_id
                      FROM students
                      WHERE mark > (SELECT AVG(Mark)
                                    FROM students); """,
                  task_5_conn)

,student_id
0,1
1,7


## Solutions task 5 (pandas)

In [0]:
students = pd.read_sql_query("SELECT * FROM students", task_5_conn)
task_5_conn.close()

In [0]:
students.loc[students.mark > students.mark.mean(), "student_id"]

0    1
6    7
Name: student_id, dtype: int64

# Task 6: Triggers

## Setup task 6

In [0]:
create_products =   """ CREATE TABLE IF NOT EXISTS products (
                                      id integer PRIMARY KEY,
                                      model text NOT NULL,
                                      price integer NOT NULL                                      
                                      );
                    """

In [0]:
insert_products =   """ INSERT INTO products (id, model, price)
                          VALUES
	                        ("1", "brand_A", "5"),
                          ("2", "brand_A", "10"),
                          ("3", "brand_B", "15");
                    """

In [0]:
create_products_history =   """ CREATE TABLE IF NOT EXISTS products_history (
                                      id integer PRIMARY KEY,
                                      product_id integer NOT NULL,
                                      model_old text NOT NULL,
                                      model_new text NOT NULL,
                                      price_old integer NOT NULL,
                                      price_new integer NOT NULL,
                                      upd_date text NOT NULL                                                                            
                                      );
                    """

In [0]:
task_6_conn = setup_task([create_products, insert_products, create_products_history])

DB setup has been successfully completed!


## Solutions task 6 (SQL)

6.1. Написать триггер, который заполняет отдельную таблицу ProductHistory данными, которые были актуальными на момент изменения информации о продукте


In [0]:
create_trigger =  """ CREATE TRIGGER keep_products_history_after_update 
                    AFTER UPDATE ON products
                    WHEN old.model <> new.model
                    OR old.price <> new.price
                      BEGIN
	                      INSERT INTO products_history (
		                      product_id,
                          model_old,
		                      model_new,
                          price_old,
                          price_new,	
		                      upd_date)
                        VALUES (
                          old.id,
                          old.model,
		                      new.model,
                          old.price,
                          new.price,		                      	
		                      DATETIME('NOW'));
                      END; """

In [0]:
task_6_conn.cursor().execute(create_trigger)

In [0]:
# check that products_history is empty
len(pd.read_sql_query("SELECT * FROM products_history", task_6_conn))

0

In [0]:
# update products by model
task_6_conn.cursor().execute("UPDATE products SET model = 'brand_A' WHERE id = 3;")

In [0]:
# check update is correct
pd.read_sql_query("SELECT * FROM products;", task_6_conn, index_col="id")

In [0]:
# check products_history;
pd.read_sql_query("SELECT * FROM products_history", task_6_conn, index_col="id")

,product_id,model_old,model_new,price_old,price_new,upd_date
id,,,,,,
1,3,brand_B,brand_A,15,15,2020-05-08 14:10:53


In [0]:
# another update by price and check
task_6_conn.cursor().execute("UPDATE products SET price = 20 WHERE model = 'brand_A';")
pd.read_sql_query("SELECT * FROM products", task_6_conn, index_col="id")

,model,price
id,,
1,brand_A,20
2,brand_A,20
3,brand_A,20


In [0]:
# check products_history;
pd.read_sql_query("SELECT * FROM products_history", task_6_conn, index_col="id")

,product_id,model_old,model_new,price_old,price_new,upd_date
id,,,,,,
1,3,brand_B,brand_A,15,15,2020-05-08 14:10:53
2,1,brand_A,brand_A,5,20,2020-05-08 14:11:00
3,2,brand_A,brand_A,10,20,2020-05-08 14:11:00
4,3,brand_A,brand_A,15,20,2020-05-08 14:11:00


In [0]:
task_6_conn.close()